In [ ]:
# results  = []
# for idx in range(n_rows):
#     row = df.iloc[idx].fillna('')
#     item = {
#         'TITLE': row['Title'],
#         'DATE' : row['Start Date'],
#         'MAINPAGE' : row['Main Page'],
#         'ADDRESS1' : row['Address1'],
#         'ADDRESS2' : row['Address2'],
#         'WEHN' : row['When'],
#         'THUMB': row['thumb'],
#         'IMAGE' : row['image'],
#         'DESC' : row['Desc'],
#         'LongDesc': ''
#     }
#     results.append(item)


In [ ]:
#from Levenshtein import ratio, jaro, distance
from collections import Counter

import pandas as pd
#import numpy  as np

import subprocess
import requests
import random
# import gzip
import json
import math
import os
import re

def set_default_path(directory: str | None = 'Event-Aggregator') -> str:
    """Changes the working directory to """
    while not os.getcwd().endswith(directory):
        os.chdir('..')
        if len(os.getcwd()) == 0:
            raise FileExistsError
    return os.getcwd()

def query(url, data, mode):
    match mode:
        case 'POST':
            item = requests.post(url,json=data)
            item = json.loads(item.content.decode())
        case 'GET':
            item = requests.get(url)
            item = json.loads(item.content.decode())
    return item

default_path    = os.path.join(set_default_path() ,'Production_Environment')
server_path     = os.path.join(default_path       ,'Servers') 
collectors_path = os.path.join(server_path        ,'Collectors')

# Events
events_path     = os.path.join(collectors_path    ,'Events')
data_path       = os.path.join(events_path        ,'Data')
File_path       = os.path.join(data_path          ,'Final_Events.csv')

# User Base
UserBase_path  = os.path.join(collectors_path ,'UserBase')
user_data_path = os.path.join(UserBase_path   ,'Data')
user_data_file = os.path.join(user_data_path  ,'User_data.json')

In [ ]:
# df = pd.read_csv(os.path.join(data_path,'Final_Events.csv'))

---

In [ ]:
# url  = "http://127.0.0.1:5005/identify"
# data = query(url = url, data = None, mode = 'GET')

In [ ]:
os.getcwd()

In [ ]:
with open(user_data_file) as f:
    data = json.load(f)

In [ ]:
sample_user = random.choice(list(data.keys()))

---

In [ ]:
url  = "http://127.0.0.1:5002/identify"
data = {
    'NUMBER': 2
}
data = query(url = url,data = data, mode = 'GET')
data

In [ ]:
url  = "http://127.0.0.1:5002/random_sample"
data = {
    'NUMBER': 2
}
data = query(url = url,data = data, mode = 'POST')
data

In [ ]:
sample_user = list(data.keys())[0]
sample_user

In [ ]:
url  = "http://127.0.0.1:5000/recc"
data = {
    'USER_ID': sample_user,
    'NUMBER': 10,
    'MODE' : 'item'
}
data = query(url = url,data = data, mode = 'POST')
data

---

In [ ]:
import scipy.sparse as sp
import numpy as np

In [ ]:
n_users = len(data.keys())
n_users

In [ ]:
n_events = 489

In [ ]:
mat = sp.dok_matrix((n_users,n_events), dtype=np.int64)
for idx, (user_id, user_data) in enumerate(data.items()):
    u_interact = user_data['CLICKED']
    for event_id, freq in u_interact.items():
        event_id = int(event_id)
        # Add our user and interactions to our mat
        mat[idx,event_id] = freq

In [ ]:

from sklearn.metrics                 import silhouette_score
from sklearn.cluster                 import KMeans

In [ ]:
X = mat.tocsr()

In [ ]:
def user_kmeans(X, n_clusters_max = 15,trials = 5) -> list[float]:
    results = []
    for n_clusters in range(2,n_clusters_max):
        c_score = 0
        for seed in range(trials):
            kmeans = KMeans(
                max_iter     = 100,
                n_clusters   = n_clusters,
                n_init       = 10,
                random_state = seed,
            ).fit(X)
            c_score += silhouette_score(X,kmeans.labels_)

        c_score /= trials
        results.append(float(c_score))
        # cluster_ids, cluster_sizes = np.unique(kmeans.labels_, return_counts=True)
        # print(f"Number of elements assigned to each cluster: {cluster_sizes}")
    return results

In [ ]:
results = user_kmeans(X)

In [ ]:
n_clusters = np.argmin(results) + 2
kmeans     = KMeans(
                max_iter     = 100,
                n_clusters   = n_clusters,
                n_init       = 20,
                random_state = 42,
            ).fit(X)

In [ ]:
entry = sp.dok_matrix((1,489), dtype=np.int8)

In [ ]:
label = kmeans.predict(entry).item()

In [ ]:
# entry.toarray()[0][0]